In [6]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pathlib import Path
# Step 1: Create a session and load the page
url1 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals'
url2 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals'
url3 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals'
url_list = [url1,url2,url3,url4]
url_list =[url +'&SeasonType=Playoffs' for url in url_list]
#url_list =[url +'&SeasonType=Regular+Season'for url in url_list]

def get_tables(url_list):
    data = []
    xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table'
    driver = webdriver.Chrome()
    for url in url_list:
        
        driver.get(url)
        print(url)
        # Wait for the page to fully load
        driver.implicitly_wait(20)
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath)))

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())

        
        df= dfs[-1]
        #print(df)
        drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        df.columns = df.columns.droplevel()
        df = df.drop(columns = drop)
        data.append(df)
    driver.close()
    return data
#url_list = [url1]#
def multiyear_shooting(url_list,team_round):
    df_list = []

    for i in range(2013,2023):
        year = i+1
        season = '&Season='+str(i)+'-'+str(i+1 - 2000)
        year_url = [url+season for url in url_list]
        year_url = [url + '&PORound='+str(team_round) for url in year_url]
        frames = get_tables(year_url)

 
        path = str(i+1)+'/playoff_shooting/round'+str(team_round)+'/'
        output_dir = Path(path)
        output_dir.mkdir(parents=True, exist_ok=True)
        #terms = ['data/teampullup.csv','data/teamcatchshoot.csv','data/teamundersix.csv','data/teamiso.csv','data/teamtransition.csv']
        terms = ['very_tight','tight','open','wide_open']
        
        
        for k in range(len(terms)):
            
            frames[k]['shot_coverage'] = terms[k]
            frames[k]['round'] = team_round
            frames[k]['year'] = year
        df = pd.concat(frames)
        df.to_csv(path+'team_shooting.csv',index = False)
        df_list.append(df)
    final_df = pd.concat(df_list)
    return final_df
year_frames = []
for i in range(1,5):
    df = multiyear_shooting(url_list,i)
    year_frames.append(df)

https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals&SeasonType=Playoffs&Season=2013-14&PORound=1
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals&SeasonType=Playoffs&Season=2013-14&PORound=1
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals&SeasonType=Playoffs&Season=2013-14&PORound=1
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals&SeasonType=Playoffs&Season=2013-14&PORound=1
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals&SeasonType=Playoffs&Season=2014-15&PORound=1
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals&SeasonType=Playoffs&Season=2014-15&PORound=1
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Op

In [7]:
master = pd.concat(year_frames)

In [8]:
master = master.sort_values(by=['year','round','shot_coverage'])
master

,TEAM,GP,G,Freq%,FGM,FGA,FG%,eFG%,2FG Freq%,2FGM,2FGA,2FG%,3FG Freq%,3PM,3PA,3P%,shot_coverage,round,year
0,Los Angeles Clippers,7,7,30.3,82,165,49.7,58.5,16.5,53,90,58.9,13.8,29,75,38.7,open,1,2014
1,San Antonio Spurs,7,7,31.5,77,162,47.5,53.7,20.6,57,106,53.8,10.9,20,56,35.7,open,1,2014
2,Atlanta Hawks,7,7,31.6,62,160,38.8,47.5,14.6,34,74,45.9,17.0,28,86,32.6,open,1,2014
3,Indiana Pacers,7,7,30.2,76,152,50.0,57.6,20.2,53,102,52.0,9.9,23,50,46.0,open,1,2014
4,Golden State Warriors,7,7,26.7,70,146,47.9,56.5,14.5,45,79,57.0,12.3,25,67,37.3,open,1,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Denver Nuggets,5,5,47.9,99,190,52.1,53.2,41.3,95,164,57.9,6.5,4,26,15.4,tight,4,2023
0,Denver Nuggets,5,5,10.1,20,40,50.0,50.0,9.8,20,39,51.3,0.3,0,1,0.0,very_tight,4,2023
1,Miami Heat,5,5,5.5,8,24,33.3,33.3,5.5,8,24,33.3,0.0,0,0,-,very_tight,4,2023
0,Miami Heat,5,5,15.5,30,68,44.1,58.1,4.1,11,18,61.1,11.4,19,50,38.0,wide_open,4,2023
